In [ ]:
import MySQLdb
from pandas.io import sql
import pandas as pd
import numpy as np
conn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="openmrs") 
openmrscursor = conn.cursor()

In [ ]:
df_order = pd.read_sql('select * from order_type', con = conn)
df_order

In [ ]:
mimicconn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="mimic4") 
mimiccursor = mimicconn.cursor()

In [ ]:
df_poe = pd.read_sql('select subject_id, hadm_id, order_type, order_subtype from poe', con = mimicconn) 

In [ ]:
df_poe = df_poe.fillna('')

In [ ]:
df_poe['ordertype'] = df_poe['order_type'] + '-' + df_poe['order_subtype']

In [ ]:
# df_pharmacy = pd.read_sql('select * from pharmacy', con = mimicconn) 
# df_pharmacy

In [ ]:
df_poe_ordertype = df_poe[['ordertype']]

In [ ]:
df_poe_ordertype.ordertype = df_poe_ordertype.ordertype.apply(lambda x: 'Medications' if 'Medications' in x else x)

In [ ]:
df_poe_ordertype = df_poe_ordertype.drop_duplicates()

In [ ]:
list(df_poe_ordertype['ordertype'])

In [ ]:
medication = df_poe_ordertype[df_poe_ordertype['ordertype'].str.contains('Medications')]
medication 

In [ ]:
df_poe_ordertype['creator']=1
df_poe_ordertype['date_created']='2022-01-01'
import uuid
df_poe_ordertype['uuid'] = [uuid.uuid4() for x in range(len(df_poe_ordertype))]
df_poe_ordertype['description'] = 'Data that is imported from mimic-iv'

In [ ]:
df_poe_ordertype

In [ ]:
# df_poe_ordertype.loc[df_poe_ordertype.ordertype == 'Medications', 'name'] = "Medication Order"

In [ ]:
# df_poe_ordertype.loc[df_poe_ordertype['name']=='Medication Order'] # confirmation

In [ ]:
df_poe_ordertype = df_poe_ordertype.rename(columns={'ordertype':'name'})
df_poe_ordertype

In [ ]:
df_poe_ordertype['java_class_name']= 'org.openmrs.TestOrder'

In [ ]:
df_poe_ordertype.loc[df_poe_ordertype.name == 'Medications', 'java_class_name'] = "org.openmrs.DrugOrder"

In [ ]:
df_poe_ordertype.loc[df_poe_ordertype['name']=='Medications'] # confirmation

In [ ]:
column_names = ['name', 'description', 'creator', 'date_created', 'uuid', 'java_class_name']
df_poe_ordertype = df_poe_ordertype.reindex(columns=column_names)

In [ ]:
# creating column list for insertion into concept table
cols = "`,`".join([str(i) for i in df_poe_ordertype.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_poe_ordertype.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `order_type` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit()

In [ ]:
df_order_type = pd.read_sql('select * from `order_type`', con = conn)
df_order_type

In [ ]:
df_order_type.loc[df_order_type['name']=='Medications'] 

# dealing with orders table

In [ ]:
df_orders = pd.read_sql('select * from `orders`', con = conn)
df_orders

In [ ]:
df_prescriptions = pd.read_sql('select * from prescriptions', con = mimicconn) 
df_prescriptions

In [ ]:
df_prescriptions_orders = df_prescriptions[['subject_id', 'hadm_id', 'pharmacy_id', 'starttime', 'stoptime', 'drug', 'dose_val_rx', 'dose_unit_rx', 'form_val_disp', 'form_unit_disp', 'doses_per_24_hrs', 'route']]
df_prescriptions_orders.isnull().sum()

In [ ]:
df_prescriptions_orders = df_prescriptions_orders[df_prescriptions_orders['drug'].notna()]


In [ ]:
drug_concept_id = pd.read_sql('select * from concept where class_id=3', con = conn)


In [ ]:
drug_concept_id['description'] = drug_concept_id['description'].str.lower()

In [ ]:
dict_drug_concept = dict(zip(drug_concept_id.description, drug_concept_id.concept_id))
len(dict_drug_concept)

In [ ]:
df_prescriptions_orders['drug'] = df_prescriptions_orders['drug'].str.lower()

In [ ]:
df_prescriptions_orders['concept_id'] = df_prescriptions_orders['drug'].map(dict_drug_concept)
df_prescriptions_orders.isnull().sum()


In [ ]:
df_prescriptions_orders[['dose_val_rx', 'dose_unit_rx', 'form_val_disp', 'form_unit_disp', 'doses_per_24_hrs', 'route']] = df_prescriptions_orders[['dose_val_rx', 'dose_unit_rx', 'form_val_disp', 'form_unit_disp', 'doses_per_24_hrs', 'route']].fillna('')

In [ ]:
def split(x):
    if '-' in x:
        return x.split('-')[1]
    else:
        return x

In [ ]:

df_prescriptions_orders.dose_val_rx = df_prescriptions_orders.dose_val_rx.apply(split)


In [ ]:
df_prescriptions_orders['instructions'] = df_prescriptions_orders['dose_val_rx'] + ' ' + df_prescriptions_orders['dose_unit_rx'] 

In [ ]:
df_prescriptions_orders['instructions'] = df_prescriptions_orders['instructions'] + ' ' + df_prescriptions_orders['drug'] + ',' + ' ' + df_prescriptions_orders['route']  

In [ ]:
df_prescriptions_orders['doses_per_24_hrs'] = df_prescriptions_orders['doses_per_24_hrs'].astype(str)

In [ ]:
df_prescriptions_orders['instructions'] = df_prescriptions_orders['instructions'] + ',' + ' ' + df_prescriptions_orders['doses_per_24_hrs'] + ' ' + 'per 24 hrs' 

In [ ]:
df_prescriptions_orders= df_prescriptions_orders.rename(columns={'subject_id':'patient_id', 'hadm_id':'encounter_id', 'pharmacy_id':'order_number', 'starttime':'date_activated', 'stoptime':'date_stopped'})

In [ ]:
hadmid_conceptid = pd.read_csv('hadm_id and concept_id.csv')
hadmid_conceptid = hadmid_conceptid.rename(columns={"hadm_id": "encounter_id", "concept_id":"indication_concept_id"})

In [ ]:
dict_ind_conceptid = dict(zip(hadmid_conceptid.encounter_id, hadmid_conceptid.indication_concept_id))
len(dict_ind_conceptid)

In [ ]:
df_prescriptions_orders['order_reason'] = df_prescriptions_orders['encounter_id'].map(dict_ind_conceptid)

In [ ]:
df_prescriptions_orders['creator'] = 1
df_prescriptions_orders['care_setting'] =1
df_prescriptions_orders['orderer'] = 1

df_prescriptions_orders['date_created']='2022-01-01'
df_prescriptions_orders['order_type_id'] =18

In [ ]:

import uuid
df_prescriptions_orders['uuid'] = [uuid.uuid4() for x in range(len(df_prescriptions_orders))]

In [ ]:
# df_prescriptions_orders = df_prescriptions_orders.rename(columns= {'indication_concept_id':'order_reason'})

In [ ]:
df_prescriptions_orders['order_action'] = 'Null'
df_prescriptions_orders = df_prescriptions_orders.replace({np.nan: None})

In [ ]:
column_names = ['order_type_id', 'concept_id', 'orderer', 'encounter_id', 'instructions', 'date_activated', 'date_stopped', 'order_reason', 'creator', 'date_created', 'patient_id', 'uuid', 'order_number', 'order_action', 'care_setting']
df_prescriptions_orders = df_prescriptions_orders.reindex(columns=column_names)

In [ ]:
df_prescriptions_orders

In [ ]:
df_prescriptions_orders1 = df_prescriptions_orders[16983509 : 17008052]
df_prescriptions_orders1

In [ ]:
df_prescriptions_orders2 = df_prescriptions_orders[3000000:6000000]
df_prescriptions_orders2

In [ ]:
df_prescriptions_orders3 = df_prescriptions_orders[5000000:10000000]
df_prescriptions_orders3

In [ ]:
df_prescriptions_orders3 = df_prescriptions_orders[10000000:15000000]

In [ ]:
df_prescriptions_orders4 = df_prescriptions_orders[15000000:17008052]

# Importing into orders table

In [ ]:
# creating column list for insertion into concept table
cols = "`,`".join([str(i) for i in df_prescriptions_orders.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_prescriptions_orders.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `orders` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit()

In [ ]:
# creating column list for insertion into concept table
cols = "`,`".join([str(i) for i in df_prescriptions_orders2.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_prescriptions_orders2.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `orders` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit()

In [ ]:
# creating column list for insertion into concept table
cols = "`,`".join([str(i) for i in df_prescriptions_orders3.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_prescriptions_orders3.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `orders` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit()

In [ ]:
# creating column list for insertion into concept table
cols = "`,`".join([str(i) for i in df_prescriptions_orders4.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_prescriptions_orders4.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `orders` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit()